In [1]:
import pandas as pd

** Step1: Explore Dataset **

In [2]:
# import dataset and check columns and observations

test_data = pd.read_csv('test_data.csv')
print(test_data.shape)

(1000000, 3)


In [3]:
test_data.head()

,code,x,y
0,7B22eXm7dO6,49.860269,75.403216
1,sb$Tb0NdrJ2,-36.011083,11.730664
2,RE3h5#ym<d9,-17.367508,96.169508
3,Dkh;g>mX.Ik,26.945207,65.115143
4,uw7X6u9$vG.,-12.547311,74.810954


In [4]:
test_data.tail()

,code,x,y
999995,Jnuo.IXPC61,13.823279,82.483209
999996,"Qw%0mqDZ,Ji",43.172851,90.463722
999997,%Z06s>#RlnC,-22.663018,11.892157
999998,VgSpr4dDgt0,45.687996,45.578465
999999,p<cg!JrlwH5,49.430680,79.457338


In [5]:
# Check if any null value

test_data.isnull().sum()

code    0
x       0
y       0
dtype: int64

** Step2: Data Transformation **

In [6]:
import re

strc = []
idx = 0

# Extract out the first occured letter: \W matches all non-alphanumeric characters
                                      # \d matches all numeric characters. 
                                      # ^ inverts the selection to match everything 
        # that is not non-alphanumeric or numeric), which corresponds to all letters.
def first_letter(row):
    match = re.compile("[^\W\d]").search(row) 
    return row[match.start()]

# Append the extracted letter into list strc and turn all the value into upper case
for row in test_data['code']:
    strc.append(first_letter(row).upper())

# Insert column "group" at the beginning of dataframe
test_data.insert(loc=idx, column='group', value=strc)

In [7]:
# Check the result
test_data.head()

,group,code,x,y
0,B,7B22eXm7dO6,49.860269,75.403216
1,S,sb$Tb0NdrJ2,-36.011083,11.730664
2,R,RE3h5#ym<d9,-17.367508,96.169508
3,D,Dkh;g>mX.Ik,26.945207,65.115143
4,U,uw7X6u9$vG.,-12.547311,74.810954


In [8]:
test_data.tail()

,group,code,x,y
999995,J,Jnuo.IXPC61,13.823279,82.483209
999996,Q,"Qw%0mqDZ,Ji",43.172851,90.463722
999997,Z,%Z06s>#RlnC,-22.663018,11.892157
999998,V,VgSpr4dDgt0,45.687996,45.578465
999999,P,p<cg!JrlwH5,49.430680,79.457338


In [16]:
# Drop the original column'code'
test_data = test_data.drop(['code'], axis=1)

In [18]:
test_data.head()

,group,x,y
0,B,49.860269,75.403216
1,S,-36.011083,11.730664
2,R,-17.367508,96.169508
3,D,26.945207,65.115143
4,U,-12.547311,74.810954


In [19]:
test_data.to_csv('test_result.csv')

** Step 3: Find slopes of fitted line and # of obs by group **

In [20]:
# Find the slopes for each group
from scipy.stats import linregress

slopes = test_data.groupby('group').apply(lambda v: linregress(v.x, v.y)[0]) #[0] only remain the slopes of fitted result
slopes

group
A    0.000431
B    0.000891
C    0.007697
D   -0.002585
E   -0.006771
F    0.009832
G   -0.001408
H    0.003276
I    0.005457
J   -0.004127
K    0.006628
L    0.009429
M    0.002112
N    0.000844
O   -0.007709
P   -0.000075
Q    0.002014
R   -0.001255
S   -0.000138
T    0.002590
U    0.002055
V    0.000453
W   -0.003936
X   -0.005922
Y   -0.008263
Z    0.003301
dtype: float64

In [22]:
# Find the # of observations for each group
counts = test_data.groupby('group').apply(len)
counts

group
A    38234
B    38405
C    38327
D    38309
E    38547
F    38704
G    38459
H    38194
I    38438
J    38413
K    38376
L    38451
M    38249
N    38855
O    38793
P    38465
Q    38547
R    38564
S    38425
T    38747
U    38515
V    38478
W    38569
X    38182
Y    38573
Z    38181
dtype: int64

** Step4: Generate final result **

In [23]:
# Join two series into one dataframe
output = pd.concat([counts, slopes], axis=1).reset_index()
output

,group,0,1
0,A,38234,0.000431
1,B,38405,0.000891
2,C,38327,0.007697
3,D,38309,-0.002585
4,E,38547,-0.006771
5,F,38704,0.009832
6,G,38459,-0.001408
7,H,38194,0.003276
8,I,38438,0.005457
9,J,38413,-0.004127


In [24]:
# Rename column names
output.rename(columns={'group': 'letter_group',0: 'counts',1: 'slopes'}, inplace=True)
output

,letter_group,counts,slopes
0,A,38234,0.000431
1,B,38405,0.000891
2,C,38327,0.007697
3,D,38309,-0.002585
4,E,38547,-0.006771
5,F,38704,0.009832
6,G,38459,-0.001408
7,H,38194,0.003276
8,I,38438,0.005457
9,J,38413,-0.004127


In [60]:
# Export result into csv file
output.to_csv('result.csv')